# Cross-Asset Hybrid Model Calibration

In this notebook, we analyse the calibration of a cross-asset hybrid model for EUR, USD and GBP. The analysis represents an extension of the 3-factor Gaussian HJM interest rate model analysis in [RatesModelCalibration](RatesModelCalibration.ipynb).

Calibration objectives are statistical properties of reference interest rates and exchange rates (or asset prices). The targeted statistical properties are volatilities and correlations of 1-month returns observed from a historical time series spanning 20 years. The data generation procedure and calculation methodology for reference volatilities and correlations are specified in the related [MarketData](https://github.com/sschlenkrich/MarketData) repository.

The calibrated model is considered a *real-world* model. The model will recover observed statistical properties. But the resulting model will a priori *not* match prices for interest rate and FX options observed in the market at a given date. Nevertheless, the calibrated model is intended to provide a starting point (or initial guess) for risk neutral calibration to interest rate option data.

An objective of the real-world calibration is to mark model parameters which are difficult to calibrate to option data. Such model parameters are mean reversion rates and instantaneous risk factor correlations.

## Packages and Preliminaries

In [ ]:
using Pkg
Pkg.activate("../.")

In [ ]:
using CSV
using DataFrames
using DiffFusion
using PlotlyJS

In [ ]:
include("utils.jl")

## Reference Rate Specification

Reference interest rates are continuously compounded zero rates.

$$
z_{t}\left(\tau\right)=-\frac{\log\left[P\left(t,t+\tau\right)\right]}{\tau}.
$$

For EUR and GBP We use rate terms $\tau \in \{ 1, 2, 5, 10, 15, 20 \}$ (measured in years). For USD we skip the 15 year reference rate term due to a lack of available historical data. Details on the reference interest rate properties are discussed in the [RatesModelCalibration](RatesModelCalibration.ipynb) notebook.

Reference exchange rates are formulated as log-prices,

$$
y_{t} = \log\left( S_t \right).
$$

## Reference Rate Data

Calibration targets are estimated from historical time series. The targets are represented by ranges for volatilities and correlations. These ranges give some indication about the uncertainty in the statistical estimation. For details, see the discussion in the [MarketData](https://github.com/sschlenkrich/MarketData) repository.

### Volatility Data

We normalise volatility data as annualised volatility and measure reference rate terms in years.

In [ ]:
file_name = "data/standard_deviation_30days.csv"
std_table = DataFrame(CSV.File(file_name))
std_table = stack(std_table, 3:size(std_table)[2])
std_table[!, "VOLATILITY"] = std_table[!, "value"] / sqrt(30.0/365.0);
std_table[!, "YEARS"] = std_table[!, "MONTHS"] / 12;

### Correlation Data

In [ ]:
file_name = "data/correlations_30days.csv"
corr_table = DataFrame(CSV.File(file_name))
corr_table = stack(corr_table, 5:size(corr_table)[2])
corr_table[!, "YEARS1"] = Int.(corr_table[!, "MONTHS1"] / 12)
corr_table[!, "YEARS2"] = Int.(corr_table[!, "MONTHS2"] / 12);

## Model Setup

For this analysis, we setup 3-factor Gaussian HJM models for EUR, USD and GBP with time-homogenous volatility parameters.

The volatility specification uses zero rate benchmark rates with terms $\{1, 10, 20\}$ (measured in years). We note that the benchmark rate terms deliberately coincide with reference rate terms. This way, we can directly control respective volatilities and correlations.

Exchange rate models are specified for USD-EUR and GBP-EUR. We use log-normal models with time-homogenous volatility parameter.

Besides the volatility parameters for interest rate and exchange rate models, we need to mark correlations between all the risk factors.

The model specification is encoded in the `model_params` dictionary below.

In [ ]:
model_params = Dict([
    # interest rate model benchmark rates, equal for all models
    (("delta_1", ""), 1.0),
    (("delta_2", ""), 10.0),
    (("delta_3", ""), 20.0),
    # interest rate model mean reversion parameters
    (("EUR_chi_1", ""), 0.01),
    (("EUR_chi_2", ""), 0.50),
    (("EUR_chi_3", ""), 0.80),
    #
    (("USD_chi_1", ""), 0.01),
    (("USD_chi_2", ""), 0.30),
    (("USD_chi_3", ""), 0.60),
    #
    (("GBP_chi_1", ""), 0.01),
    (("GBP_chi_2", ""), 0.20),
    (("GBP_chi_3", ""), 0.40),
    # interest rate model volatlity parameters
    (("EUR_f_1", ""), 0.0060),
    (("EUR_f_2", ""), 0.0068),
    (("EUR_f_3", ""), 0.0071),
    #
    (("USD_f_1", ""), 0.0070),
    (("USD_f_2", ""), 0.0085),
    (("USD_f_3", ""), 0.0080),
    #
    (("GBP_f_1", ""), 0.0078),
    (("GBP_f_2", ""), 0.0086),
    (("GBP_f_3", ""), 0.0077),
    # exchange rate modelvolatility parameters
    (("USD-EUR_x", ""), 0.090),
    (("GBP-EUR_x", ""), 0.075),
    # interest rate model correlations
    (("EUR_f_1", "EUR_f_2"), 0.55),
    (("EUR_f_2", "EUR_f_3"), 0.95),
    (("EUR_f_1", "EUR_f_3"), 0.45),
    #
    (("USD_f_1", "USD_f_2"), 0.55),
    (("USD_f_2", "USD_f_3"), 0.95),
    (("USD_f_1", "USD_f_3"), 0.45),
    #
    (("GBP_f_1", "GBP_f_2"), 0.60),
    (("GBP_f_2", "GBP_f_3"), 0.95),
    (("GBP_f_1", "GBP_f_3"), 0.50),
    # rates vs. FX correlations
    (("EUR_f_1", "USD-EUR_x"), 0.25),
    (("EUR_f_2", "USD-EUR_x"), 0.10),
    (("EUR_f_3", "USD-EUR_x"), 0.08),
    #
    (("USD_f_1", "USD-EUR_x"), -0.15),
    (("USD_f_2", "USD-EUR_x"), -0.10),
    (("USD_f_3", "USD-EUR_x"), -0.08),
    #
    (("GBP_f_1", "USD-EUR_x"), 0.05),
    (("GBP_f_2", "USD-EUR_x"), -0.05),
    (("GBP_f_3", "USD-EUR_x"), -0.07),
    #
    (("EUR_f_1", "GBP-EUR_x"), 0.07),
    (("EUR_f_2", "GBP-EUR_x"), 0.03),
    (("EUR_f_3", "GBP-EUR_x"), 0.02),
    #
    (("USD_f_1", "GBP-EUR_x"), -0.18),
    (("USD_f_2", "GBP-EUR_x"), -0.22),
    (("USD_f_3", "GBP-EUR_x"), -0.20),
    #
    (("GBP_f_1", "GBP-EUR_x"), -0.25),
    (("GBP_f_2", "GBP-EUR_x"), -0.17),
    (("GBP_f_3", "GBP-EUR_x"), -0.13),
    # FX vs. FX correlation
    (("USD-EUR_x", "GBP-EUR_x"), 0.35),
    # rates vs. rates correlations
    (("EUR_f_1", "USD_f_1"), 0.55),
    (("EUR_f_1", "USD_f_2"), 0.45),
    (("EUR_f_1", "USD_f_3"), 0.35),
    #
    (("EUR_f_2", "USD_f_1"), 0.40),
    (("EUR_f_2", "USD_f_2"), 0.75),
    (("EUR_f_2", "USD_f_3"), 0.75),
    #
    (("EUR_f_3", "USD_f_1"), 0.35),
    (("EUR_f_3", "USD_f_2"), 0.70),
    (("EUR_f_3", "USD_f_3"), 0.75),
    #
    (("EUR_f_1", "GBP_f_1"), 0.67),
    (("EUR_f_1", "GBP_f_2"), 0.47),
    (("EUR_f_1", "GBP_f_3"), 0.37),
    #
    (("EUR_f_2", "GBP_f_1"), 0.45),
    (("EUR_f_2", "GBP_f_2"), 0.80),
    (("EUR_f_2", "GBP_f_3"), 0.75),
    #
    (("EUR_f_3", "GBP_f_1"), 0.37),
    (("EUR_f_3", "GBP_f_2"), 0.75),
    (("EUR_f_3", "GBP_f_3"), 0.75),
    #
    (("USD_f_1", "GBP_f_1"), 0.55),
    (("USD_f_1", "GBP_f_2"), 0.40),
    (("USD_f_1", "GBP_f_3"), 0.35),
    #
    (("USD_f_2", "GBP_f_1"), 0.45),
    (("USD_f_2", "GBP_f_2"), 0.77),
    (("USD_f_2", "GBP_f_3"), 0.72),
    #
    (("USD_f_3", "GBP_f_1"), 0.32),
    (("USD_f_3", "GBP_f_2"), 0.77),
    (("USD_f_3", "GBP_f_3"), 0.75),    
]);

The parameters in the dictionary can be modified. And the impact on model-implied quantities can be observed in below output graphs.

### Using DiffFusion Example Model

Alternatively, we may use a pre-defined DiffFusion Example model. For this use case, we specify the model name in the model parameters variable.

The pre-defined model must be a EUR-based model with context and correlations consistent with this analysis.

We use this functionality to double-check that the pre-defined DiffFusion Example model `g3_3factor_real_world` is consistent with our calibrated model from this analysis.

In [ ]:
# model_params = "g3_3factor_real_world"

## Model Analysis

In below graphs we compare ranges (box plots) of estimates for historical volatility and correlation with corresponding model-implied quantities.

Model-implied volatilities and correlations are evaluated using the methodology implemented in `DiffFusion.reference_rate_volatility_and_correlation(...)`.

Model-implied volatilities and correlations for 1-year, 10-year, 20-year rates and exchange rates can be controlled effectively by the corresponding input model parameters. Other model-implied quantities are *interpolated* by the model.

In [ ]:
update_hybrid_plots!(
    [
    ],
    model_params, std_table, corr_table,
    plot_vols = true,
    plot_rates_corrs = true,
    plot_fx_corrs = true,
    plot_fx_rates_corrs = true,
    plot_rates_rates_corrs = true,
    scaling_type = DiffFusion.ZeroRateScaling,
)

The model input parameters can be modified to successively match historical volatilities and correlations for the various models.

It turns out that the most challenging part is the interest rate model calibration. The methodology for interest rate model calibration is discussed in [RatesModelCalibration](RatesModelCalibration.ipynb).

We find that we can match well the structure of interest rate and exchange rate volatilities and correlations for various reference terms. The analysis demonstrates that the chosen modelling setup is well suited to model historically observed statistical properties of interest rates and exchange rates.

## Model Analysis for 1-Factor Rates Models

We repeat above analysis and test whether we can also match reference volatilities and correlations with a setting where interest rates are modelled with only a for 1-factor models.

For this exercise, we switch-off *zero rate scaling* of volatilities and set volatility parameters for second and third interest rate factor to zero. This way, the 3-factor model degenerates to a 1-factor model.

In [ ]:
model_params_1f = Dict([
    # interest rate model benchmark rates, equal for all models
    (("delta_1", ""), 1.0),
    (("delta_2", ""), 10.0),
    (("delta_3", ""), 20.0),
    # interest rate model mean reversion parameters
    (("EUR_chi_1", ""), 0.01),
    (("EUR_chi_2", ""), 0.50),
    (("EUR_chi_3", ""), 0.80),
    #
    (("USD_chi_1", ""), 0.01),
    (("USD_chi_2", ""), 0.30),
    (("USD_chi_3", ""), 0.60),
    #
    (("GBP_chi_1", ""), 0.01),
    (("GBP_chi_2", ""), 0.20),
    (("GBP_chi_3", ""), 0.40),
    # interest rate model volatlity parameters
    (("EUR_f_1", ""), 0.0075),
    (("EUR_f_2", ""), 0.0000),
    (("EUR_f_3", ""), 0.0000),
    #
    (("USD_f_1", ""), 0.0090),
    (("USD_f_2", ""), 0.0000),
    (("USD_f_3", ""), 0.0000),
    #
    (("GBP_f_1", ""), 0.0090),
    (("GBP_f_2", ""), 0.0000),
    (("GBP_f_3", ""), 0.0000),
    # exchange rate modelvolatility parameters
    (("USD-EUR_x", ""), 0.090),
    (("GBP-EUR_x", ""), 0.075),
    # interest rate model correlations
    # rates vs. FX correlations
    (("EUR_f_1", "USD-EUR_x"), 0.10),
    #
    (("USD_f_1", "USD-EUR_x"), -0.15),
    #
    (("GBP_f_1", "USD-EUR_x"), -0.05),
    #
    (("EUR_f_1", "GBP-EUR_x"), 0.02),
    #
    (("USD_f_1", "GBP-EUR_x"), -0.20),
    #
    (("GBP_f_1", "GBP-EUR_x"), -0.15),
    # FX vs. FX correlation
    (("USD-EUR_x", "GBP-EUR_x"), 0.35),
    # rates vs. rates correlations
    (("EUR_f_1", "USD_f_1"), 0.55),
    #
    (("EUR_f_1", "GBP_f_1"), 0.60),
    #
    (("USD_f_1", "GBP_f_1"), 0.55),
]);
#
update_hybrid_plots!(
    [
    ],
    model_params_1f, std_table, corr_table,
    plot_vols = true,
    plot_rates_corrs = true,
    plot_fx_corrs = true,
    plot_fx_rates_corrs = true,
    plot_rates_rates_corrs = true,
    scaling_type = DiffFusion.DiagonalScaling,
)

As expected, the 1-factor interest rate model setting does not allow to match observed volatility structures and correlation patterns.